In [ ]:
import requests
import pandas as pd
import datetime
import time
import json

In [ ]:
api_user = "INSERT_API_USER"
api_pass = "INSERT_API_KEY|1"
headers = {"content-type": "application/json"}

In [ ]:
intro_url = f"https://api.securevan.com/v4/apiKeyProfiles"
response = (
    requests.get(
    intro_url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
        ).json()
    )

In [ ]:
response

In [ ]:
res_url = f"https://api.securevan.com/v4/changedEntityExportJobs/resources"

entities = (
    requests.get(
    res_url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
        ).json()
    )

In [ ]:
entities

In [ ]:
contacts_fields_url = f"https://api.securevan.com/v4/changedEntityExportJobs/fields/contacts"
contacts_fields = (
    requests.get(
    contacts_fields_url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
        ).json()
    )

In [ ]:
all_fields = [x["fieldName"] for x in contacts_fields]

In [ ]:
all_fields

In [ ]:
from_timestamp = (datetime.datetime.now() - datetime.timedelta(days=30)).isoformat()

post_cee_job_url = f"https://api.securevan.com/v4/changedEntityExportJobs/"

request_body = json.dumps(
    {
        "dateChangedFrom": from_timestamp,
        "resourceType": "contacts",
        # "requestedIds": ["10162461"],
        "requestedFields": all_fields,
        "fileSizeKbLimit": 100000
        }
    )


In [ ]:
request_body

In [ ]:
posted_job_response = (
    requests.post(post_cee_job_url,
    headers=headers,
    data=request_body,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
            ).json()
    )

posted_job_response

In [ ]:
export_job_id = posted_job_response["exportJobId"]

job_check_url = f"https://api.securevan.com/v4/changedEntityExportJobs/{export_job_id}"

while True:
    job_check_response = (
        requests.get(job_check_url,
        headers=headers,
        auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
                ).json()
        )
    if job_check_response["jobStatus"] != "Complete":
        print(f"Job Status: {job_check_response['jobStatus']}")
        time.sleep(3)
    else:
        print(f"Job Status: {job_check_response['jobStatus']}")
        print(f"File at {job_check_response['files'][0]['downloadUrl']}")
        break

In [ ]:
job_check_response

In [ ]:
file_url = job_check_response["files"][0]["downloadUrl"]
resp = requests.get(file_url)
with open("response_file.csv", "wb") as f:
    f.write(resp.content)

# file access expires after 24 hours
frame = pd.read_csv("response_file.csv")
frame